In [1]:
import pandas as pd
import numpy as np
import cv2
import importlib
SEED = 1234
np.random.seed(SEED) 

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.applications.vgg16 import VGG16
import mylibs.ResNet as ResNet
import mylibs.SENet as SENet
importlib.reload(ResNet)
importlib.reload(SENet)
from keras.models import Model

from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [ ]:
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session  
# config = tf.ConfigProto()  
# config.gpu_options.allow_growth = True
# set_session(tf.Session(config=config)) 

In [ ]:
%cd E:\kaggle\iceberg

In [ ]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

def get_more_images(imgs):
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
    vh_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        vert_flip_imgs.append(cv2.flip(imgs[i], 1))
        hori_flip_imgs.append(cv2.flip(imgs[i], 0))
        vh_flip_imgs.append(cv2.flip(imgs[i], -1))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
    vh = np.array(vh_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h, vh))
    
    return more_images

In [ ]:
use_custom_augmentation = True
if use_custom_augmentation:
    df_train = pd.read_json('E:/kaggle/iceberg/train.json/data/processed/train.json')
    df_test = pd.read_json('E:/kaggle/iceberg/test.json/data/processed/test.json')
    Xtrain = get_scaled_imgs(df_train)
    Xtest = get_scaled_imgs(df_test)
    Ytrain = np.array(df_train['is_iceberg'])
    
    df_train["inc_angle"] = df_train["inc_angle"].replace('na',0)
    df_test["inc_angle"] = df_test["inc_angle"].replace('na',0)
    idx_tr = np.where(df_train["inc_angle"]>0)
    Xtrain = Xtrain[idx_tr[0]]
    Ytrain = Ytrain[idx_tr[0]]
    
    Xtrain = get_more_images(Xtrain) 
    Ytrain = np.concatenate((Ytrain,Ytrain,Ytrain, Ytrain))

In [ ]:
# VGG16, ResNet50, SENet50
baseModelName = "SENet50"
def getModel(baseModelName):
    if baseModelName == "VGG16":
        baseModel = VGG16(weights='imagenet', include_top=False, input_shape=Xtrain.shape[1:], pooling = "avg")
        cnnOutput = baseModel.output
    elif baseModelName == "ResNet50":
        baseModel = ResNet.ResNet50(weights='imagenet', include_top=False, input_shape=Xtrain.shape[1:], pooling = "avg")
        cnnOutput = baseModel.output
    elif baseModelName == "SENet50":
        baseModel = SENet.SENet50(weights='imagenet', include_top=False, input_shape=Xtrain.shape[1:], pooling = "avg",
                                  kernel_regularizer = "l2",
                                  bias_regularizer = "l2",
                                  activity_regularizer = None,
                                  regularizer_value = 1e-4)
        cnnOutput = baseModel.output

    if baseModelName == "VGG16":
        fcOutput = Dropout(0.6)(cnnOutput)
        predictions = Dense(1, activation="sigmoid")(fcOutput)
    elif baseModelName == "ResNet50":
        predictions = Dense(1, activation="sigmoid")(cnnOutput)
    elif baseModelName == "SENet50":
        predictions = Dense(1, activation="sigmoid")(cnnOutput)
    
    model = Model(inputs=baseModel.input, outputs=predictions)
    
    optimizer = Adam(lr=1e-4)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [ ]:
model = getModel(baseModelName)
model.summary()
plot_model(model, to_file="senet50-reg.png")

In [ ]:
# for layer in model.layers:
#     print(layer.name, layer.trainable)

In [ ]:
batch_size = 64
earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

In [ ]:
K=3
Kfolds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=SEED).split(Xtrain, Ytrain))
y_test_pred_log = 0
for j, (train_idx, test_idx) in enumerate(Kfolds):
    print('\n===================FOLD=',j)
    Xtrain_cv = Xtrain[train_idx]
    Ytrain_cv = Ytrain[train_idx]
    Xtrain_val = Xtrain[test_idx]
    Ytrain_val = Ytrain[test_idx]
    
    model_file = 'model_%s.hdf5' % j
    
    mcp_save = ModelCheckpoint(model_file, save_best_only=True, monitor='val_loss', mode='min')
    model = getModel(baseModelName)
    
    model.fit(Xtrain_cv, Ytrain_cv, batch_size=batch_size, epochs=100, verbose=1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss, TensorBoard(log_dir='./logs')], validation_data=(Xtrain_val, Ytrain_val))

    model.load_weights(filepath = model_file)    
    
    score = model.evaluate(Xtrain, Ytrain, verbose=1)
    print('Train score:', score[0])
    print('Train accuracy:', score[1])
    y_test_pred_log += model.predict(Xtest).reshape(Xtest.shape[0])
    
y_test_pred_log /= K

In [ ]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': y_test_pred_log})
print(submission.head(10))
print(submission.count(), Xtest.shape[0])

submission.to_csv('submission-cnn-custom-%s.csv' % baseModelName, index=False)